### Importação dos metadados

In [4]:
from src import transformer
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import nltk
%load_ext autoreload
%autoreload 2
df = pd.read_csv("dados/corpus_tratado/metadados.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Filtrando os ids por assuntos

In [5]:
corte = 50
df_validos = pd.DataFrame(df.groupby("Assunto").size()).reset_index()
df_validos.columns = ["assunto", "quant"]
df_validos = df_validos.loc[df_validos.quant >= corte]
display(df_validos.quant.sum())
display(df_validos.assunto.nunique())
documentos_validos = df[df.Assunto.isin(df_validos.assunto)][["id", "Assunto"]].reset_index().drop('index', axis = 1)
diretorio = "dados/corpus_tratado/"
documentos_validos['arquivo'] = documentos_validos.id + '.txt'
documentos_validos['teores'] = [transformer.recuperar_teor(x, diretorio) for x in tqdm(documentos_validos['arquivo'])]
documentos_validos

35027

125

,id,Assunto,arquivo,teores
0,1500075-43.2019.8.26.0569,Tráfico de Drogas e Condutas Afins,1500075-43.2019.8.26.0569.txt,trata-se de denúncia oferecida pelo ministério...
1,1019468-32.2019.8.26.0562,Alienação Fiduciária,1019468-32.2019.8.26.0562.txt,aymoré crédito financiamento e investimento s ...
2,0000043-68.2019.8.26.0616,Furto,0000043-68.2019.8.26.0616.txt,matheus cardoso de siqueira rosa qualificado n...
3,1007259-02.2018.8.26.0292,DIREITO PREVIDENCIÁRIO,1007259-02.2018.8.26.0292.txt,francisco joaquim dos santos ajuizou a present...
4,1000425-53.2019.8.26.0128,Rural (Art. 48/51),1000425-53.2019.8.26.0128.txt,odair lacerda devidamente qualificado nos auto...
...,...,...,...,...
35022,0001233-88.2018.8.26.0426,Cheque,0001233-88.2018.8.26.0426.txt,0 caracterizada a hipótese do artigo 0 ii do c...
35023,1514316-66.2019.8.26.0037,Impostos,1514316-66.2019.8.26.0037.txt,ante o requerimento do exequente noticiando qu...
35024,1008737-24.2019.8.26.0320,Indenização por Dano Material,1008737-24.2019.8.26.0320.txt,dispensado o relatório nos termos do artigo 0 ...
35025,0000206-22.2019.8.26.0075,IPTU/ Imposto Predial e Territorial Urbano,0000206-22.2019.8.26.0075.txt,0 - tendo em vista a comprovação de depósito e...


### cortando documentos curtos

In [7]:
min_palavras_documento = 50
documentos_validos['validos'] = documentos_validos.teores.apply(lambda x: 0 if len(x.split(' ')) <= min_palavras_documento else 1)
documentos_validos = documentos_validos[documentos_validos['validos'] == 1].drop(['arquivo', 'teores', 'validos'], axis = 1).reset_index().drop('index', axis = 1)
documentos_validos

,id,Assunto
0,1500075-43.2019.8.26.0569,Tráfico de Drogas e Condutas Afins
1,1019468-32.2019.8.26.0562,Alienação Fiduciária
2,0000043-68.2019.8.26.0616,Furto
3,1007259-02.2018.8.26.0292,DIREITO PREVIDENCIÁRIO
4,1000425-53.2019.8.26.0128,Rural (Art. 48/51)
...,...,...
33308,0001233-88.2018.8.26.0426,Cheque
33309,1514316-66.2019.8.26.0037,Impostos
33310,1008737-24.2019.8.26.0320,Indenização por Dano Material
33311,0000206-22.2019.8.26.0075,IPTU/ Imposto Predial e Territorial Urbano


### Execução do fluxo de experimentação

In [ ]:
n_experimentos = 1
lista_k = np.arange(2,201)
grid_minfreqs = [0, 50, 100]
grid_stopwords = [True, False]
grid_ica = [True, False]
grid_tesauro = [True, False]
grid_dimensoes = [100,300]
df = transformer.transform_param(documentos_validos, n_experimentos, grid_minfreqs, grid_stopwords, grid_ica, grid_tesauro, grid_dimensoes, lista_k)

Importando w2v


2021-04-05 21:16:33,675 : INFO : loading projection weights from modelos/100/w2v_skip_nilc.txt
2021-04-05 21:17:54,436 : INFO : loaded (929606, 100) matrix from modelos/100/w2v_skip_nilc.txt


Importando ftt


2021-04-05 21:17:54,438 : INFO : loading projection weights from modelos/100/ftt_skip_nilc.txt
2021-04-05 21:19:12,495 : INFO : loaded (929605, 100) matrix from modelos/100/ftt_skip_nilc.txt


Importando glv


...


calculando a similaridade entre assuntos para o modelo vec_ftt_ger_soma



--------- Agrupando dados para o modelo vec_glv_ger_soma no experimento__minfreq_0__com_crit_tesauro__com_crit_ica__manteve_sw_pt__100_dims__1 ---------



--------- executando analyzer para experimento __minfreq_0__com_crit_tesauro__com_crit_ica__manteve_sw_pt__100_dims__1 ---------
calculando matriz de similaridade nos vetores vec_glv_ger_soma



calculando a similaridade entre assuntos para o modelo vec_glv_ger_soma



--------- Agrupando dados para o modelo vec_w2v_jur_soma no experimento__minfreq_0__com_crit_tesauro__com_crit_ica__manteve_sw_pt__100_dims__1 ---------


In [ ]:
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
lista_k = [2, 3, 4]
plt.rcParams.update(plt.rcParamsDefault)
dir_experimento = 'experimento___minfreq_0__com_crit_tesauro__com_crit_ica__removeu_sw_pt__100_dims__1'
le = LabelEncoder()
modelo = 'vec_w2v_ger_soma'
df = pd.read_csv('dados/'+dir_experimento+'/vetores_teste.csv')
df[modelo] = df[modelo].apply(lambda x: transformer.converter_string_array(x))
X_kmeans = np.stack(df[modelo])
X_kmeans = X_kmeans.reshape(X_kmeans.shape[0], X_kmeans.shape[2])
y_kmeans = df['assunto']
le.fit(y_kmeans)
y_kmeans = le.transform(y_kmeans)
lista_scores_k = transformer.computar_scores_agrupamento(X_kmeans, y_kmeans, dir_experimento, modelo, lista_k)
transformer.gerar_graficos_kmeans(lista_scores_k, dir_experimento, modelo)
np.save(dir_experimento + '/lista_scores_k.npy', lista_scores_k)